# 1. Building powerful image classification models using very little data


## 1.1 A Small ConvNet 
As a baseline

In [ ]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/media/ubuntu16/Documents/datasets/dogs-vs-cats-2000/train/'
validation_data_dir = '/media/ubuntu16/Documents/datasets/dogs-vs-cats-2000/validation'

nb_train_samples = 2000
nb_validation_samples = 800

epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

# generate batches of image data (and their labels) directly from our jpgs in their respective folders.
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

#model.save_weights('first_try.h5')

## 1.2 Fine-tuning a pre-trained network

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import optimizers


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/media/ubuntu16/Documents/datasets/dogs-vs-cats-2000/train/'
validation_data_dir = '/media/ubuntu16/Documents/datasets/dogs-vs-cats-2000/validation/'

nb_train_samples = 2000
nb_validation_samples = 800

epochs = 50
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

# generate batches of image data (and their labels) directly from our jpgs in their respective folders.
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# 构建不带分类器的预训练模型
# 因不包含top层，输入维度不用固定
base_model = VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# 
x = base_model.output 
x = GlobalAveragePooling2D()(x)
#x = Flatten(input_shape=base_model.output_shape[1:])(x)

# 添加全连接层
x = Dense(256, activation='relu', name='dense_1')(x)

# 添加分类器 (二分类使用sigmoid, 多分类使用softmoax)
predictions = Dense(1, activation='sigmoid', name='output')(x)

# 构建完整模型
model = Model(input=base_model.input, output=predictions)

# 首先，锁住预训练好的卷积层， 仅训练顶部的两层
for i, layer in enumerate(base_model.layers):
    layer.trainable = False
    print(i,layer.name)
    
# 编译模型，一定要在锁层以后操作
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

# 在新的数据集上训练
hist_top = model.fit_generator(train_generator,
                               samples_per_epoch=nb_train_samples//batch_size,
                               epochs = epochs,
                               verbose = 1,
                               validation_data = validation_generator,
                               validation_steps= nb_validation_samples//batch_size)


# 微调VGG16部分卷积层
# 通过查看各层编号和名字，锁住前15层，训练之后的层
# 不能微调所有层，因为整个网络的表示能力过于强大，很容易过拟合
for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True
    
# 重新编译模型，使设置生效 （注意使用sgd）
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

# fine-tune the model
hist_tune = model.fit_generator(train_generator,
                               samples_per_epoch=nb_train_samples//batch_size,
                               epochs = epochs,
                               verbose = 1,
                               validation_data = validation_generator,
                               validation_steps= nb_validation_samples//batch_size)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,1, figsize=(10, 20))
axes[0].plot(hist_top.history['loss'], label='train')
axes[0].plot(hist_top.history['val_loss'], label='validation')

axes[1].plot(hist_tune.history['loss'], label='train')
axes[1].plot(hist_tune.history['val_loss'], label='validation')

plt.legend()
plt.show()


In [ ]:
from keras.applications.vgg16 import VGG16 
base_model = VGG16(weights='imagenet', include_top=False)

print(len(base_model.layers))
print()
print(base_model.inputs) # 模型输入张量的列表，可能由多个输入
print(base_model.input)
print()
print(base_model.outputs) # 模型输入张量的列表，可能由多个输入
print(base_model.output)
print()
print(base_model.input_shape)
print(base_model.output_shape)
print()
print(base_model.summary())

In [ ]:
base_model.get_config()

# Multi-GPU


In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
 
from __future__ import print_function
 
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
 
batch_size = 128
num_classes = 10
epochs = 20

num_gpu=8
 
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
 
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# float64 is not supported in GPU
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
 
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

try:
    parallel_model = multi_gpu_model(model, gpus=num_gpu, cpu_relocation=True)
    print("Training using multiple GPUs..")
except ValueError:
    parallel_model = model
    print("Training using single GPU or CPU..")
    
 
 
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer=RMSprop(),
                       metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='acc', save_best_only=True, verbose=1)

callbacks = [model_checkpoint]
history = parallel_model.fit(x_train, y_train,
                             batch_size=batch_size,
                             epochs=epochs,
                             verbose=1,
                             validation_data=(x_test, y_test),
                             callbacks=callbacks)

best_model = load_model('best_model.h5')
score = best_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Training using single GPU or CPU..
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
6

In [ ]:
1. 使用 float64, 显存占满，GPU利用率40%左右
2. 使用float32, 显存占满，GPU利用率44%左右